# Projet: Débordement d'égouts
## Rapport équipe 9 - A19
#### Noboru Yoshida - Mehdi Chaid - Mathieu Giroud-Huppé - Maxime Gosselin
#### 05 Décembre 2020

## Table des matières

1. [Objectif](#objectives)  
2. [Méthode choisie](#chosen_method)
3. [Traitement des données](#data_processing)  
    1. [Chargement des données de l'ensemble d'entraînement](#data_loading) 
    2. [Nettoyage des données](#data_cleaning)  
    3. [Augmentation des données](#data_augmentation)
4. [Analyse exploratoire](#data_analysis)  
    1. [Distance à la station météorologique](#distance-meteo) 
    2. [Hauteur du trop plein](#very-full-height)
    3. [TODO: Continuer](#continue)
5. [Sélection de modèles](#model-selection)
    1. [Régression logistique](#logistic-reg)
    2. [Classification bayésienne naive](#naive-bayes)
    3. [Forêt aléatoire](#random-forest)
    4. [Séparateur à vaste marge](#svm)
6. [Améliorations possibles](#possible-improvements)

<a id="objectives"></a>

## 1. Objectif

Ce rapport vise à fournir une prédiction sur les surverses dans plusieurs ouvrages sur l'île de Montréal. TODO: Continuer

<a id="chosen_method"></a>

## 2. Méthode choisie

TODO

<a id="data_processing"></a>

## 3. Traitement des données

TODO

<a id="data_analysis"></a>

## 4. Analyse exploratoire


